In [20]:
import PIL
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler


We propose a method for editing images from human instructions: given an input image and a written instruction that tells the model what to do, our model follows these instructions to edit the image. To obtain training data for this problem, we combine the knowledge of two large pretrained models---a language model (GPT-3) and a text-to-image model (Stable Diffusion)---to generate a large dataset of image editing examples. Our conditional diffusion model, InstructPix2Pix, is trained on our generated data, and generalizes to real images and user-written instructions at inference time. Since it performs edits in the forward pass and does not require per-example fine-tuning or inversion, our model edits images quickly, in a matter of seconds. We show compelling editing results for a diverse collection of input images and written instructions.

In [21]:
model_id = "timbrooks/instruct-pix2pix"
# pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, safety_checker=None)
pipe.enable_sequential_cpu_offload()
#pipe.to("cuda")
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)



text_encoder\model.safetensors not found
Fetching 15 files: 100%|██████████| 15/15 [00:00<00:00, 555.60it/s]


In [22]:
torch.cuda.empty_cache()

In [23]:
image = PIL.Image.open("judios.jpg")

In [24]:
prompt = "make it look like a statue"
images = pipe(prompt, image=image, num_inference_steps=10, image_guidance_scale=1).images[0]
#show the image in the notebook
images.show()


 10%|█         | 1/10 [12:34<1:53:08, 754.25s/it]


KeyboardInterrupt: 

In [ ]:
images.save(prompt[:10] + ".jpg")

In [ ]:
from PIL import Image

def reduce_image_quality(input_path, output_path, quality=20):
    """
    Reduce the quality of an image to lower its file size.

    Parameters:
    - input_path (str): Path to the input image file.
    - output_path (str): Path to save the image with reduced quality.
    - quality (int): The quality of the image (0-100). Higher values mean better quality.
    """
    try:
        # Open the image file
        with Image.open(input_path) as img:
            # Save the image with reduced quality
            img.save(output_path, quality=quality)

        print(f"Image with reduced quality saved to {output_path}")

    except Exception as e:
        print(f"Error: {e}")



# Example usage:
input_image_path = "jorge.jpg"
output_image_path = "low_quality_image.jpg"
reduce_image_quality(input_image_path, output_image_path, quality=2)


Image with reduced quality saved to low_quality_image.jpg
